In [1]:
import warnings
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, RobustScaler, MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from category_encoders import MEstimateEncoder, OneHotEncoder, PolynomialEncoder, LeaveOneOutEncoder
import os
import numpy as np
import pandas as pd
import time

warnings.filterwarnings('ignore')

In [2]:
train_path = os.path.join("..", "data", "input", "train_treated.csv")
val_path = os.path.join("..", "data", "input", "val_treated.csv")
test_path = os.path.join("..", "data", "input", "test_treated.csv")

X_train = pd.read_csv(train_path, index_col=0)
X_val = pd.read_csv(val_path, index_col=0)
test = pd.read_csv(test_path, index_col=0)

In [3]:
X_train.drop(['primary_diagnosis', 'secondary_diagnosis',
             'additional_diagnosis'], axis=1, inplace=True)
X_val.drop(['primary_diagnosis', 'secondary_diagnosis',
           'additional_diagnosis'], axis=1, inplace=True)
test.drop(['primary_diagnosis', 'secondary_diagnosis',
          'additional_diagnosis'], axis=1, inplace=True)

In [4]:
X_train.shape

(56988, 67)

In [5]:
y = pd.read_csv("../data/input/target.csv", index_col=0)
y['readmitted_binary'].values

array(['No', 'No', 'No', ..., 'Yes', 'No', 'No'], dtype=object)

In [6]:
X = pd.concat([X_train, X_val], axis=0)

numerical_features = X.select_dtypes(include='number').columns.tolist()
metric_features = [
    feature for feature in numerical_features if not feature.startswith('med_')]
metric_features

['outpatient_visits_in_previous_year',
 'emergency_visits_in_previous_year',
 'inpatient_visits_in_previous_year',
 'average_pulse_bpm',
 'length_of_stay_in_hospital',
 'number_lab_tests',
 'non_lab_procedures',
 'number_of_medications',
 'number_diagnoses',
 'outpatient_visits_in_previous_year_log',
 'emergency_visits_in_previous_year_log',
 'inpatient_visits_in_previous_year_log',
 'length_of_stay_in_hospital_log',
 'non_lab_procedures_log',
 'number_of_medications_log',
 'number_diagnoses_log',
 'outpatient_visits_in_previous_year.1',
 'emergency_visits_in_previous_year.1',
 'inpatient_visits_in_previous_year.1',
 'average_pulse_bpm.1',
 'length_of_stay_in_hospital.1',
 'number_lab_tests.1',
 'non_lab_procedures.1',
 'number_of_medications.1',
 'number_diagnoses.1',
 'age_mean']

In [9]:
X[metric_features].describe()

,outpatient_visits_in_previous_year,emergency_visits_in_previous_year,inpatient_visits_in_previous_year,average_pulse_bpm,length_of_stay_in_hospital,number_lab_tests,non_lab_procedures,number_of_medications,number_diagnoses,outpatient_visits_in_previous_year_log,...,outpatient_visits_in_previous_year.1,emergency_visits_in_previous_year.1,inpatient_visits_in_previous_year.1,average_pulse_bpm.1,length_of_stay_in_hospital.1,number_lab_tests.1,non_lab_procedures.1,number_of_medications.1,number_diagnoses.1,age_mean
count,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,...,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000
mean,0.369588,0.196249,0.640154,99.611222,4.391024,43.095654,1.340923,15.995452,7.421023,-523.259114,...,0.327924,0.164327,0.620417,99.611222,4.391024,43.040120,1.340923,15.926961,7.417865,66.461340
std,1.287469,0.910854,1.267271,23.040521,2.988739,19.642919,1.706664,8.122347,1.937809,231.537851,...,0.908966,0.523021,1.143228,23.040521,2.988739,19.508325,1.706664,7.774923,1.918082,15.673493
min,0.000000,0.000000,0.000000,60.000000,1.000000,1.000000,0.000000,1.000000,1.000000,-625.632573,...,0.000000,0.000000,0.000000,60.000000,1.000000,1.000000,0.000000,3.000000,2.000000,5.000000
25%,0.000000,0.000000,0.000000,80.000000,2.000000,31.000000,0.000000,10.000000,6.000000,-625.632573,...,0.000000,0.000000,0.000000,80.000000,2.000000,31.000000,0.000000,10.000000,6.000000,55.000000
50%,0.000000,0.000000,0.000000,100.000000,4.000000,44.000000,1.000000,15.000000,8.000000,-625.632573,...,0.000000,0.000000,0.000000,100.000000,4.000000,44.000000,1.000000,15.000000,8.000000,65.000000
75%,0.000000,0.000000,1.000000,119.000000,6.000000,57.000000,2.000000,20.000000,9.000000,-625.632573,...,0.000000,0.000000,1.000000,119.000000,6.000000,57.000000,2.000000,20.000000,9.000000,75.000000
max,42.000000,76.000000,21.000000,139.000000,14.000000,121.000000,6.000000,75.000000,16.000000,1.693188,...,5.000000,3.000000,6.000000,139.000000,14.000000,85.000000,6.000000,43.000000,9.000000,95.000000


In [7]:
cat_features = [
    feature for feature in X.columns if feature not in numerical_features]
cat_features

['race',
 'gender',
 'age',
 'payer_code',
 'admission_type',
 'discharge_disposition',
 'admission_source',
 'glucose_test_result',
 'a1c_test_result',
 'change_in_meds_during_hospitalization',
 'prescribed_diabetes_meds',
 'is_outpatient_visited',
 'is_emergency_visited',
 'is_inpatient_visited',
 'is_pulse_normal',
 'primary_diagnosis_cat',
 'secondary_diagnosis_cat',
 'additional_diagnosis_cat',
 'discharge_disposition_cat',
 'admission_source_cat']

In [8]:
def avg_score(scaler):
    # apply kfold
    skf = StratifiedKFold(n_splits=10)
    model = DecisionTreeClassifier(
        criterion='gini',
        max_depth=15,
        random_state=42
    )
    # create lists to store the results from the different models
    score_train = []
    score_test = []
    timer = []
    f1_s = []

    for train_index, test_index in skf.split(X, y):
        # get the indexes of the observations assigned for each partition
        X_train, X_val = X[metric_features].iloc[train_index], X[metric_features].iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]

        scale = scaler.fit(X_train)
        # Transform your train data by applying the scale obtained in the previous command
        scaled_X_train = scale.transform(X_train)
        # Transform your validation data by applying the scale obtained in the first command
        scaled_X_val = scale.transform(X_val)

        oe = OrdinalEncoder()

        X_cat_train = oe.fit_transform(X[cat_features].iloc[train_index])
        X_cat_val = oe.fit_transform(X[cat_features].iloc[test_index])

        scaled_X_train = np.concatenate((scaled_X_train, X_cat_train), axis=1)
        scaled_X_val = np.concatenate((scaled_X_val, X_cat_val), axis=1)

        # start counting time
        begin = time.perf_counter()
        # fit the model to the data
        model.fit(scaled_X_train, y_train)
        # finish counting time
        end = time.perf_counter()
        # check the mean accuracy for the train
        value_train = model.score(scaled_X_train, y_train)
        # check the mean accuracy for the test
        value_test = model.score(scaled_X_val, y_val)
        # check the f1 score
        y_pred = model.predict(scaled_X_val)
        value_f1 = f1_score(y_val, y_pred, pos_label='Yes')
        # append the accuracies, the time and the number of iterations in the corresponding list
        score_train.append(value_train)
        score_test.append(value_test)
        timer.append(end-begin)
        f1_s.append(value_f1)
    # calculate the average and the std for each measure (accuracy, time and number of iterations)
    avg_time = round(np.mean(timer), 3)
    avg_train = round(np.mean(score_train), 3)
    avg_test = round(np.mean(score_test), 3)
    std_time = round(np.std(timer), 2)
    std_train = round(np.std(score_train), 2)
    std_test = round(np.std(score_test), 2)
    avg_f1 = round(np.mean(f1_s * 100), 6)
    std_f1 = round(np.std(f1_s * 100), 6)

    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train), \
        str(avg_test) + '+/-' + str(std_test), str(avg_f1) + '+/-' + str(std_f1)


def show_results(df, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    for arg in args:
        # obtain the results provided by avg_score
        time, avg_train, avg_test, f1 = avg_score(arg)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_test, f1
        count += 1
    return df

In [9]:
results_empty = pd.DataFrame(columns=['Time', 'Train', 'Test', 'f1'], index=[
                             'MinMax[0, 1]', 'MinMax[-1, 1]', 'StandardScaler', 'Robust', 'Abs'])

results = show_results(results_empty,
                       MinMaxScaler(feature_range=(0, 1)),
                       MinMaxScaler(feature_range=(-1, 1)),
                       StandardScaler(),
                       RobustScaler(),
                       MaxAbsScaler())

results

,Time,Train,Test,f1
"MinMax[0, 1]",2.559+/-0.39,0.902+/-0.0,0.856+/-0.03,0.05501+/-0.028038
"MinMax[-1, 1]",2.697+/-0.05,0.902+/-0.0,0.856+/-0.03,0.055138+/-0.028032
StandardScaler,2.238+/-0.62,0.902+/-0.0,0.856+/-0.03,0.055137+/-0.028028
Robust,1.575+/-0.63,0.902+/-0.0,0.856+/-0.03,0.054965+/-0.028023
Abs,2.688+/-0.05,0.902+/-0.0,0.856+/-0.03,0.055127+/-0.028027


since the best results where form `MinMaxScaler[-1, 1]`, we will use that scaler


In [10]:
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
test_scaled = test.copy()

standard_scaler = StandardScaler()

X_train_scaled[metric_features] = standard_scaler.fit_transform(
    X_train[metric_features])
X_val_scaled[metric_features] = standard_scaler.transform(
    X_val[metric_features])
test_scaled[metric_features] = standard_scaler.transform(test[metric_features])

## encoding


### research

after a research found this options the best for the following type of variables:

- **binary**: binary encoder (dah!) <br/>
- **imbalanced binary**: m-estimate encoder (probably test vs binary) <br/>
- **low cardinallity**: one hot encoding vs target encoding (wins if var is corr with target variable), polynominal encoder worth looking <br/>
- **high cardinallity**: base n encoder (potentially leave one out) <br/>


now we will classify each variable


In [11]:
def describe_categorical(features, dataframe):
    # Initialize lists to store data for each column in the report
    feature_list = []
    mode_list = []
    mode_freq_list = []
    mode_prop_list = []
    second_mode_list = []
    second_mode_freq_list = []
    second_mode_prop_list = []
    missing_val_percent_list = []
    cardinality_list = []

    for feature in features:
        # Calculate mode, 2nd mode and their frequencies
        mode = dataframe[feature].mode()[0]
        mode_freq = dataframe[feature].value_counts().iloc[0]
        mode_prop = mode_freq / len(dataframe)
        second_mode = dataframe[feature].value_counts().index[1] if len(
            dataframe[feature].value_counts()) > 1 else 'N/A'
        second_mode_freq = dataframe[feature].value_counts().iloc[1] if len(
            dataframe[feature].value_counts()) > 1 else 0
        second_mode_prop = second_mode_freq / len(dataframe)

        # Calculate missing values percentage and cardinality
        missing_val_percent = dataframe[feature].isna().mean() * 100
        cardinality = dataframe[feature].nunique()

        # Append to lists
        feature_list.append(feature)
        mode_list.append(mode)
        mode_freq_list.append(mode_freq)
        mode_prop_list.append(mode_prop)
        second_mode_list.append(second_mode)
        second_mode_freq_list.append(second_mode_freq)
        second_mode_prop_list.append(second_mode_prop)
        missing_val_percent_list.append(missing_val_percent)
        cardinality_list.append(cardinality)

    # Create the DataFrame
    categorical_data_quality_report = pd.DataFrame({
        'Feature': feature_list,
        'Mode': mode_list,
        'Mode Frequency': mode_freq_list,
        'Mode Proportion': mode_prop_list,
        '2nd Mode': second_mode_list,
        '2nd Mode Frequency': second_mode_freq_list,
        '2nd Mode Proportion': second_mode_prop_list,
        'Missing Values %': missing_val_percent_list,
        'Cardinality': cardinality_list
    })

    return categorical_data_quality_report.sort_values(by=['Mode Proportion', 'Missing Values %'], ascending=False)

In [12]:
cat_info = describe_categorical(cat_features, X).set_index("Feature")
cat_info.sort_values('Cardinality')

,Mode,Mode Frequency,Mode Proportion,2nd Mode,2nd Mode Frequency,2nd Mode Proportion,Missing Values %,Cardinality
Feature,,,,,,,,
is_emergency_visited,False,63242,0.887781,True,7994,0.112219,0.0,2
is_outpatient_visited,False,59587,0.836473,True,11649,0.163527,0.0,2
prescribed_diabetes_meds,Yes,54890,0.770537,No,16346,0.229463,0.0,2
is_inpatient_visited,False,47231,0.663022,True,24005,0.336978,0.0,2
is_pulse_normal,True,36338,0.510107,False,34898,0.489893,0.0,2
gender,Female,38231,0.536681,Male,33005,0.463319,0.0,2
change_in_meds_during_hospitalization,No,38326,0.538014,Ch,32910,0.461986,0.0,2
glucose_test_result,none,67548,0.948228,Norm,1806,0.025352,0.0,4
a1c_test_result,none,59320,0.832725,>8,5705,0.080086,0.0,4


In [13]:
bool_features = cat_info[(cat_info['Cardinality'] == 2) & (
    cat_info['Mode Proportion'] <= 0.6)].index.tolist()
imb_bool_features = cat_info[(cat_info['Cardinality'] == 2) & (
    cat_info['Mode Proportion'] > 0.6)].index.tolist()
low_card_features = cat_info[(cat_info['Cardinality'] > 2) & (
    cat_info['Cardinality'] <= 10)].index.tolist()
high_card_features = cat_info[cat_info['Cardinality'] > 10].index.tolist()

print(bool_features)
print(imb_bool_features)
print(low_card_features)
print(high_card_features)

['change_in_meds_during_hospitalization', 'gender', 'is_pulse_normal']
['is_emergency_visited', 'is_outpatient_visited', 'prescribed_diabetes_meds', 'is_inpatient_visited']
['glucose_test_result', 'a1c_test_result', 'race', 'discharge_disposition_cat', 'admission_source_cat', 'admission_type', 'age']
['discharge_disposition', 'admission_source', 'payer_code', 'secondary_diagnosis_cat', 'primary_diagnosis_cat', 'additional_diagnosis_cat']


#### bool encoding


In [28]:
oe = OrdinalEncoder()

X_train_encoded = X_train_scaled.copy()
X_val_encoded = X_val_scaled.copy()
test_encoded = test_scaled.copy()

X_train_encoded[bool_features] = oe.fit_transform(
    X_train_encoded[bool_features])
X_val_encoded[bool_features] = oe.transform(X_val_encoded[bool_features])
test_encoded[bool_features] = oe.transform(test_encoded[bool_features])

In [29]:
X_encoded = pd.concat([X_train_encoded, X_val_encoded], axis=0)

y = y.reindex(X_encoded.index)

In [30]:
y["readmitted_binary"] = oe.fit_transform(y)

In [31]:
not_imb_features = [
    feature for feature in cat_features if feature not in imb_bool_features]
X_encoded[not_imb_features] = oe.fit_transform(X_encoded[not_imb_features])
X_encoded[imb_bool_features].head()

,is_emergency_visited,is_outpatient_visited,prescribed_diabetes_meds,is_inpatient_visited
encounter_id,,,,
672135,False,False,Yes,False
794587,False,False,Yes,False
694232,False,False,No,False
305869,False,False,No,True
181753,False,False,Yes,False


In [18]:
for m_value in [0.1, 0.5, 1.0, 2.0]:
    m_estimate_encoder = MEstimateEncoder(cols=imb_bool_features, m=m_value)
    X_res = m_estimate_encoder.fit_transform(X_encoded, y)

    model = LogisticRegression(random_state=69)
    scores = cross_val_score(model, X_res, y, cv=10, scoring='f1')

    print(f'm = {m_value}, Mean Accuracy: {scores.mean()}')

m = 0.1, Mean Accuracy: 0.01362842669692455
m = 0.5, Mean Accuracy: 0.013634571033627077
m = 1.0, Mean Accuracy: 0.013880903700351313
m = 2.0, Mean Accuracy: 0.01388214592480264


In [32]:
# for imbalanced features
mee = MEstimateEncoder(verbose=1, cols=imb_bool_features, m=0.5)

X_train_encoded[imb_bool_features] = mee.fit_transform(
    X_train_encoded[imb_bool_features], y.loc[X_train_encoded.index])
X_val_encoded[imb_bool_features] = mee.transform(
    X_val_encoded[imb_bool_features])
test_encoded[imb_bool_features] = mee.transform(
    test_encoded[imb_bool_features])

#### low and high cardinality encoding


In [33]:
# should move this into a function
X_encoded = pd.concat([X_train_scaled, X_val_scaled], axis=0)

not_low_card = [
    feature for feature in cat_features if feature not in low_card_features]
X_encoded[not_low_card] = oe.fit_transform(X_encoded[not_low_card])

ohe = OneHotEncoder(cols=low_card_features, verbose=1,
                    use_cat_names=True, handle_unknown='value')
pe = PolynomialEncoder(cols=low_card_features, verbose=1)

for encoder in [ohe, pe]:
    res = encoder.fit_transform(X_encoded, y)

    model = LogisticRegression(random_state=42)
    scores = cross_val_score(model, res, y, cv=10, scoring='f1')

    print(f'm = {type(encoder).__name__}, Mean Accuracy: {scores.mean()}')

m = OneHotEncoder, Mean Accuracy: 0.016554182565134072
m = PolynomialEncoder, Mean Accuracy: 0.015579957424571874


In [34]:
pe = PolynomialEncoder(verbose=1, cols=low_card_features)

X_train_encoded = pe.fit_transform(
    X_train_encoded, y.loc[X_train_encoded.index])
X_val_encoded = pe.transform(X_val_encoded)
test_encoded = pe.transform(test_encoded)

In [35]:
X_train_encoded.drop(['intercept'], axis=1, inplace=True)
X_val_encoded.drop(['intercept'], axis=1, inplace=True)
test_encoded.drop(['intercept'], axis=1, inplace=True)

In [36]:
X_encoded = pd.concat([X_train_scaled, X_val_scaled], axis=0)

not_high_card = [
    feature for feature in cat_features if feature not in high_card_features]
X_encoded[not_high_card] = oe.fit_transform(X_encoded[not_high_card])

looe05 = LeaveOneOutEncoder(cols=high_card_features, verbose=1, sigma=0.05)
looe10 = LeaveOneOutEncoder(cols=high_card_features, verbose=1, sigma=0.10)
looe25 = LeaveOneOutEncoder(cols=high_card_features, verbose=1, sigma=0.25)
looe50 = LeaveOneOutEncoder(cols=high_card_features, verbose=1, sigma=0.50)
looe60 = LeaveOneOutEncoder(cols=high_card_features, verbose=1, sigma=0.60)
pe = PolynomialEncoder(cols=high_card_features, verbose=1)

for encoder in [looe05, looe10, looe25, looe50, looe60, pe]:
    res = encoder.fit_transform(X_encoded, y)

    model = LogisticRegression(random_state=69)
    scores = cross_val_score(model, res, y, cv=10, scoring='f1')

    print(f'm = {type(encoder).__name__}, Mean Accuracy: {scores.mean()}')

ohe = OneHotEncoder(cols=low_card_features, verbose=1,
                    use_cat_names=True, handle_unknown='value')
pe = PolynomialEncoder(cols=low_card_features, verbose=1)

m = LeaveOneOutEncoder, Mean Accuracy: 0.022542078743958093
m = LeaveOneOutEncoder, Mean Accuracy: 0.01941548404800995
m = LeaveOneOutEncoder, Mean Accuracy: 0.015811375896053095
m = LeaveOneOutEncoder, Mean Accuracy: 0.010423764431786576
m = LeaveOneOutEncoder, Mean Accuracy: 0.015606917866610009
m = PolynomialEncoder, Mean Accuracy: 0.021121523131579095


In [37]:
looe = LeaveOneOutEncoder(cols=high_card_features, verbose=1, sigma=0.05)

X_train_encoded = looe.fit_transform(
    X_train_encoded, y.loc[X_train_encoded.index])
X_val_encoded = looe.transform(X_val_encoded)
test_encoded = looe.transform(test_encoded)

In [38]:
X_train_encoded.to_csv("../data/input/train_encoded.csv")
X_val_encoded.to_csv("../data/input/val_encoded.csv")
test_encoded.to_csv("../data/input/test_encoded.csv")
y.to_csv("../data/input/y_bin.csv")